### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%capture
!pip install transformers[torch]
!pip install transformers datasets

In [ ]:
from pathlib import Path
from datasets import load_dataset, load_metric
import numpy as np
import pickle
import scipy as sp
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification, BertTokenizer, EarlyStoppingCallback, Trainer, TrainingArguments

### Load dataset

In [ ]:
raw_datasets = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## BERT training -- based on HuggingFace tutorial, with Trainer class

### Prepare datasets

In [ ]:
def tokenize_function(examples):
  # tokenize and replace page breaking signs
  return tokenizer([i.replace("<br />", "") for i in examples["text"]], padding="max_length", return_tensors="pt", truncation=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

### Train models

In [ ]:
metric = load_metric('accuracy')

def compute_accuracy(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-10-c9beb0262ea1>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
training_params = {
    "learning_rate": 2e-05,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 5e-06,
    "max_steps": 3500,
    "evaluation_strategy": "steps",
    "eval_steps": 500,
    "save_steps": 1000,
    "load_best_model_at_end": True,
}
early_stopping_patience=1000

In [ ]:
model_path = Path("/content/drive/MyDrive/models_new/bert_imdb/full_trainer_20230912_custom_training_params_3")

In [ ]:
training_args = TrainingArguments(model_path, **training_params)
trainer_full = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_accuracy,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
)
trainer_full.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.316800,0.211346,0.918400
1000,0.240600,0.239908,0.921120
1500,0.222300,0.203125,0.931640
2000,0.149400,0.235149,0.933280
2500,0.134400,0.256136,0.926920
3000,0.131600,0.238516,0.936080
3500,0.080500,0.264640,0.937680


TrainOutput(global_step=3500, training_loss=0.18221623338971818, metrics={'train_runtime': 10481.7858, 'train_samples_per_second': 5.343, 'train_steps_per_second': 0.334, 'total_flos': 1.473000932327424e+16, 'train_loss': 0.18221623338971818, 'epoch': 2.24})

In [ ]:
trainer_full.model.save_pretrained(model_path / "_final")